In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

In [2]:
output_notebook()

Loading BokehJS ...

In [51]:
import pandas as pd

from bokeh.models import LinearAxis, Range1d, LabelSet, ColumnDataSource
from bokeh.layouts import gridplot, row

In [11]:
from math import sqrt

In [37]:
%ls

test-lb-policy.csv   test.csv             visualization.ipynb


In [38]:
df = pd.read_csv('test.csv')
df_lb = pd.read_csv('test-lb-policy.csv')

In [39]:
df['std_dev'] = df.var_resp_time.apply(sqrt)

In [40]:
df['small_load'] = df.small_arrival / 18.73
df_lb['small_load'] = df.small_arrival / 18.73

In [56]:
p = figure(plot_width=400, plot_height=400, title="Mean response time")
source = ColumnDataSource(df)

p.line(df.small_load, df.avg_resp_time, line_color='navy')
p.line(df_lb.small_load, df_lb.avg_resp_time, line_color='navy',line_dash="4 4", legend='LB initial policy')
p.line(df_lb.small_load, 0.9*df_lb.avg_resp_time, line_color='gray',line_dash="4 4", legend='Constraint')
p.circle(df.small_load, df.avg_resp_time, color='white', line_color='navy',size=5)

labels = LabelSet(x="small_load", y="avg_resp_time", text="beta", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)
p.grid.grid_line_dash = [2,2]
p.grid.grid_line_alpha = 0.1

p.xaxis.axis_label = 'Small cell load'
p.legend.location  = 'bottom_right'

show(p) # show the results

In [52]:
p2 = figure(plot_width=400, plot_height=400, title="Average Power")

p2.line(df.small_load, df.avg_power, line_color='firebrick')
p2.line(df_lb.small_load, df_lb.avg_power, line_color='firebrick',line_dash="4 4", legend='LB initial policy')
p2.circle(df.small_load, df.avg_power, color='white', line_color='firebrick',size=5)

labels = LabelSet(x="small_load", y="avg_power", text="beta", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')

p2.add_layout(labels)
p2.grid.grid_line_dash = [2,2]
p2.grid.grid_line_alpha = 0.1

p2.xaxis.axis_label = 'Small cell load'
p2.legend.location  = 'bottom_right'

show(p2) # show the results

In [57]:
grid = gridplot([[p,p2]])

show(grid)

In [58]:
%ls


Four_small_cells_initial_plot.png  test.csv
test-lb-policy.csv                 visualization.ipynb
test-test.csv


In [60]:
tst = pd.read_csv('test-test.csv', sep='\t')

In [67]:
s = tst.loc[ 0, 'Macro-Small']

In [83]:
mac = s.split(',')[0][1:]
sma = s.split(',')[1][1:-1]

In [86]:
int(mac)

1004

In [87]:
int(sma)

0